# Import

In [ ]:
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import keras_tuner as kt

     |████████████████████████████████| 98 kB 3.5 MB/s 


# Loading Data

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between (img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.00
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Base Model

In [ ]:
def model_builder():
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units=512, activation='relu'))
  model.add(keras.layers.Dense(10))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
model = model_builder()
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)


Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5757 - accuracy: 0.8018 - val_loss: 0.4972 - val_accuracy: 0.8244
Epoch 2/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4489 - accuracy: 0.8410 - val_loss: 0.4291 - val_accuracy: 0.8552
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4139 - accuracy: 0.8515 - val_loss: 0.4100 - val_accuracy: 0.8563
Epoch 4/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4204 - accuracy: 0.8500 - val_loss: 0.4940 - val_accuracy: 0.8365
Epoch 5/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4005 - accuracy: 0.8569 - val_loss: 0.4443 - val_accuracy: 0.8446
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3888 - accuracy: 0.8602 - val_loss: 0.4200 - val_accuracy: 0.8577
Epoch 7/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3800 - accuracy: 0.8655 - val_loss: 0.4463 - val_accuracy:

# Keras Tuner

In [ ]:
def build_tune_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units=512, activation='relu'))
  model.add(keras.layers.Dense(units=hp.Int('units', min_value=16, max_value=512, step=16), activation='relu')) # tune dense layer neurons count
  model.add(keras.layers.Dense(10))

  # tune learning rate
  alpha = hp.Choice('learning_rate', values=[0.01, 0.005, 0.001, 0.0005, 0.0001])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=alpha),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model


In [ ]:
tuner = kt.Hyperband(hypermodel=build_tune_model,
                     objective="val_accuracy",
                     max_epochs=30,
                     overwrite=True,
                     directory="./result",
                     )

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 512, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.005, 0.001, 0.0005, 0.0001], 'ordered': True}


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(img_train, label_train, batch_size=128, validation_split=0.2, callbacks=[stop_early])

Trial 90 Complete [00h 00m 52s]
val_accuracy: 0.8347499966621399

Best val_accuracy So Far: 0.8939999938011169
Total elapsed time: 00h 19m 55s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./result/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 96
learning_rate: 0.005
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 16c71f938447dd919fca8e314de56910
Score: 0.8939999938011169
Trial summary
Hyperparameters:
units: 416
learning_rate: 0.01
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: f322c773e74a813af9c4d375d6133be6
Score: 0.893583357334137
Trial summary
Hyperparameters:
units: 272
learning_rate: 0.005
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: ce6f905131bc7a9b22e288f741dd8c3d
Score: 0.8932499885559082
Trial summary
Hyperparameters:
units: 352
learning_rate: 0.005
tuner/epochs: 30
tuner/initial_epoch: 10
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 50b79776dc30e4581c5891da8f81e88f
Score: 0.8929166793823242
Trial summary
Hyperparameters:
unit

In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 96)                49248     
                                                                 
 dense_2 (Dense)             (None, 10)                970       
                                                                 
Total params: 452,138
Trainable params: 452,138
Non-trainable params: 0
_________________________________________________________________
